# NCBIのintron_featureのBEDファイルを取得し、可視化を行う
**目的**<br>
スプライスによって切り出され得るイントロンの遺伝子上での分布を確認<br>
イントロンの起点と終点などの情報とともに可視化<br>
他の諸情報も同時に表示<br>

## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
from lib.geneinfo import gene_id

# モジュールのインポート
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
gene_id('TTN')

In [ ]:
gene_id = '7273'

In [ ]:
# intron_file = '../data/intron/mouse_ttn_intron.tsv'
# gbk_file = '../data/gbk/mouse_ttn.gb'
intron_file = '../data/intron/human_ttn_intron.tsv'
gbk_file = f'../data/gbk/{gene_id}.gb'

## 変数の設定

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
# gbk.read_gbk(gbk_file)
gbk.get_gbk('7273')

In [ ]:
# 当該遺伝子の染色体上の位置を確認
with open(gbk_file,'r') as f:
    for line in f:
        if "ACCESSION" in line:
            acc = line.split()[3]
            break
start, end = map(int, (acc.replace("complement(","").replace(")","")).split(".."))
print(start, end)

## intron_featurファイルを読み込み、データフレーム化

In [ ]:
# データの読み込み
bed = pd.read_csv(intron_file, sep='\t', header=None, usecols=[1,2,5])

# メジャーなストランドの確認
major_strand = bed[5].value_counts().idxmax()

# メジャーなストランドのみを抽出
bed = bed[bed[5] == major_strand]
bed = bed.loc[:,[1,2]]

# 遺伝子領域外のイントロンを除外
bed = bed[(start < bed[1]) & (bed[2] < end)]

#
if major_strand == '-':
    bed = bed.apply(lambda x: end-x)
    bed = bed.loc[:,[2,1]]
    bed.columns = ["start","end"]
    bed = bed.sort_values('start')
else:
    bed = bed.apply(lambda x: x-start+1)
    bed.columns = ["start","end"]
    bed = bed.sort_values('start')

In [ ]:
# イントロンの位置の辞書を作成
intron_dic = {}
for n,l in enumerate(gbk.intron_list()):
    intron_dic[n+1] = l

In [ ]:
# イントロの位置情報をもとに、イントロン番号を取得
def start_pos(x):
    for k,v in intron_dic.items():
        if x == v[0]:
            return k
def end_pos(x):
    for k,v in intron_dic.items():
        if x == v[1]:
            return k

## 上記情報をもとにデータフレームの作成

In [ ]:
bed['start_pos'] = bed['start'].apply(start_pos)
bed['end_pos'] = bed['end'].apply(end_pos)
bed = bed.dropna()
bed['start_pos'].astype(int)
bed['end_pos'].astype(int)
bed['intron_skip'] = bed['end_pos'] - bed['start_pos']

In [ ]:
# 5'側を起点にイントロンの組み合わせの数をカウント
s = bed.value_counts('start_pos').to_dict()
for i in range(gbk.intron_num()):
    if i+1 not in s.keys():
        s[i+1] = 0
sdf = pd.DataFrame.from_dict(s, orient='index').sort_index()

# 3'側を起点にイントロンの組み合わせの数をカウント
e = bed.value_counts('end_pos').to_dict()
for i in range(gbk.intron_num()):
    if i+1 not in e.keys():
        e[i+1] = 0
edf = pd.DataFrame.from_dict(e, orient='index').sort_index()

# エクソン配列が3の倍数であるエクソンの表示
exon_3x = []
for i in range(gbk.exon_num()):
    if len(gbk.exon_seq(i+1)) % 3 == 0:
        exon_3x.append(1)
    else:
        exon_3x.append(0)
df_3x = pd.DataFrame({"exon":[i+1 for i in range(gbk.exon_num())], 
                   "existence":exon_3x, 
                   "length":[len(gbk.exon_seq(i+1)) for i in range(gbk.exon_num())], 
                   "bar": [1 for i in range(gbk.exon_num())]})

# イントロンのスプライスバリアントの作成
skip_bed = bed[bed['intron_skip'] > 1]
skip_bed = skip_bed.sort_values('intron_skip', ascending=False)
skip_bed = skip_bed.sort_values('start_pos')

numlst = [[i,j] for i,j in zip(skip_bed['start_pos'], skip_bed['end_pos'])]
result = {0:[numlst[0]]}

for i in numlst[1:]:
    for j in range(len(result.keys())):
        if result[j][-1][1] < i[0]:
            result[j].append(i)
            break
    else:
        result.setdefault(max(result.keys())+1,[i])           
x_arr = np.array([])
y_arr = np.array([])
for i in range(len(result.keys())):
    x_arr = np.concatenate([x_arr, np.array( [i + [None] for i in result[i]]).flatten()])
    y_arr = np.concatenate([y_arr, np.full(3*len(result[i]), i)])

## 上記情報の可視化

In [ ]:
# グラフ作成
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig.add_trace(go.Bar(x=df_3x['exon'], y=df_3x['existence'], hovertemplate='Intron%{x}<extra></extra>'), row=1, col=1)
fig.add_trace(go.Bar(x=sdf.index, y=sdf[0], hovertemplate='Intron%{x}<extra></extra>'), row=2, col=1)
fig.add_trace(go.Scatter(x=x_arr, y=y_arr, hovertemplate='Intron edge: Intron%{x}<extra></extra>'), row=3, col=1)
fig.add_trace(go.Bar(x=edf.index, y=edf[0], hovertemplate='Intron%{x}<extra></extra>'), row=4, col=1)

# 各サブプロットのタイトルを追加
x = 0
titles = ["3の倍数の塩基数のエクソン", "スプライスされたイントロンの起点の数", "スプライスされたイントロンの長さ", "スプライスされたイントロンの終点の数"]
y_pos = [1.03, 0.755, 0.465, 0.175]
for t, y in zip(titles, y_pos):
    fig.add_annotation(text=f"{t}", xref="paper", yref="paper", x=x, y=y, showarrow=False, font=dict(size=12))
fig.update_layout(width=1200, height=800, title_text=f"各種情報の比較 (NM_001267550.2)", showlegend=False)

fig.show()